In [ ]:
#Este procedimento deverá ficar automatico. 
#Fazer a copia do BD 
#Tratar os dados categoricos 
#Tratar valores nulos

#Para isso vamos utilizar o apache airflow para orquestrar estas tarefas

#no AnacondaPrompt (C:\ProgramData\Anaconda3\bin) ir na pasta do airflow 
#C:\Users\gusta\airflow    #Windows
#/home/airflow             #Linux 

#rodar os comandos

#-- Iniciando o banco de dados
#airflow db init

#-- cria o usuário que será utilizado para entrar na página do Apache-Airflow e verificar as dags instanciadas

#(Substitua o colchete com o seu nome)
#airflow users create --username admin --password admin --firstname [nome] --lastname
#[sobrenome] --role Admin --email [seu e-mail]

#copiar o arquivo data_pipeline.py para a pasta dags no diretorio: 
#C:\Users\gusta\airflow\dags      #Windows
#/home/airflow/dags               #Linux 


#O arquivo data_pipeline é o ETL do modelo, que será execultado de acordo com o agendamento feito no airflow, 
#ou seja o airflow é o orquestrador de código python

#após copiar o arquivo data_pipeline.py para a pasta 
#C:\Users\gusta\airflow\dags      #Windows
#/home/airflow/dags               #Linux 

#rodar os seguintes comandos 

#-- Iniciando o servidor web   (usar um terminal Linux)
#airflow webserver


#-- Iniciando o agendador de tarefas  (usar outro terminal Linux)
#airflow scheduler


#apos startar o airflow, ir no navegador web e carregar a pagina do airflow local: digitar -> localhost:8080  

#https://medium.com/analytics-vidhya/getting-started-with-airflow-b0042cae3ebf#:~:text=Airflow%20requires%20a%20home%20directory,time%20you%20open%20a%20terminal.&text=Set%20variables%20for%20Airflow%20and%20Python%20version.


In [ ]:
#url = "C://Users//gusta//Meu Drive//Colab Notebooks//DML//3.2-DML-Deploy_ML_SQLite-Pycaret-Airflow-Streamlit_api_streamlit//data//imoveis.csv"  #Windows

url = "/home/gustavo/Documentos/api_streamlit/data/imoveis.csv"  #Linux

data = pd.read_csv(url)

data.head()

In [ ]:
# Exibindo as informações da base de dados

data.info()

In [ ]:
# Análise Estatística Básica.

data.describe()

In [ ]:
# Verificando a correlação dos atributos

plt.figure(figsize =(8,8))
cor = data.corr()
sns.heatmap(cor,annot =True)

In [ ]:
# Visualizando as correlações em pares

sns.pairplot(data)

In [ ]:
#!pip install sweetviz  #fazer a instalação via prompt do Anaconda 

In [ ]:
import sweetviz as sv

my_report = sv.analyze(data, "valor_aluguel")
my_report.show_html()

In [ ]:
my_report.show_notebook()

In [ ]:
# # Como os valores de aluguel estão distribuidos?

# Intalando o plotly para visualizações dinamicas

#'pip install plotly'

import plotly.express as px
df = px.data.tips()
fig = px.histogram(data, x="valor_aluguel", marginal="rug", hover_data=data.columns)
fig.show()

In [ ]:
import plotly.express as px
fig = px.box(data, x="valor_aluguel", points="all")
fig.show()

In [ ]:
# Alguns insigths identificados...
# 1. Dados com uma assimetria positiva. 
# 2. Possíveis outliers acima de 15.000,00. 
# 3. A maioria dos valores estão na faixa de R$ 499 à 10.000,00.

# To see the total column more clearly see the plot given below this confirms our inference about the data

plt.figure(figsize =(12,5))
sns.rugplot(
            data['valor_aluguel'],
            height=0.6,    
            )

In [ ]:
# * Existe qualquer relação entre o número de quartos e o valor do aluguel?

sns.jointplot(y= data['valor_aluguel'],x = data['num_quartos'],kind ='reg')

# * Podemos observar que o número de quartos influencia no valor do aluguel.
# * Os maiores imoveis mais caros são com números de quartos 3 e 4.
# * Como uma regra geral temos uma relação positiva entre o número de imóveis e o valor do aluguel.

In [ ]:
# # O que podemos observar sobre a possibilidade de aceitar animais?

sns.countplot(x = data['aceita_animais']
              ,hue = data['cidade'])

# * Podemos observar que o número de imóveis que aceitam animais é bem maior do que os que não aceitam.

In [ ]:
data["animal_categorical"] =  data['aceita_animais'].map({1:"aceita",0:"nao_aceita"})


sns.boxplot( x = data['valor_aluguel']
            ,y = data['animal_categorical'])


# * A maioria dos imóveis que aceitam animais, custam mais caros do que os que não aceitam.

In [ ]:
# # Cidades e os números de quartos

pd.DataFrame(data['cidade'].unique())

sns.catplot(x='cidade', y='num_quartos', data =data)

# * Rio de Janeiro é a cidade com o maior número de quartos.

In [ ]:
# # Como estão distribuidos os imóveis por número de banheiros?

plt.subplot(2,1,1)
sns.countplot(x = data['num_banheiros'])

# * A maioria dos imóveis contém de 1 a 2 banheiros.
# * Existem poucos imóveis com 5 ou mais banheiros.

In [ ]:
sns.catplot(x ='num_banheiros', y ='valor_aluguel', data = data)

# * O maior aluguel é de um imóvel que tem 7 banheiros.
# * Existem alugueis caros com imóveis com 1 a 4 banheiros.

In [ ]:
# # A mobília nos imoveis impactam no valor do aluguel?


data["mobilia_categorical"] =  data['mobilia'].map({1:"mobiliado",0:"nao_mobiliado"})


sns.boxplot( x = data['valor_aluguel']
            ,y = data['mobilia_categorical'])


# * De todos os imóveis com valores maiores de aluguel, bem acima da média, são todos não mobiliados.
# * O que justifica uma fraca correlação entre os atributos.

In [ ]:
# # Existe alguma relação entre o número de vagas de garagem e o valor do aluguel?

plt.figure(figsize =(6,4))
ax = sns.regplot(x = data['garagem'],y = data['valor_aluguel'])

In [ ]:
plt.figure(figsize =(6,4))
sns.displot(x = data['garagem'], kde = True)

# Existe uma pequena correlação entre o número de vagas de garagem e o valor do aluguel.

In [ ]:
# # Vamos analisar o seguro incêndio

sns.violinplot(x ='cidade', y ='valor_seguro_incendio', data = data,hue ='cidade')

# * São Paulo é a cidade com os maiores valores de seguro incêndio, seguido por Porto Alegre, Belo Horizonte, Rio de Janeiro e Campinas.

In [ ]:
ax = sns.regplot(x = data['valor_seguro_incendio'],y = data['valor_aluguel'])

# * Existe uma alta correlação entre o valor do seguro incendio e o valor do aluguel. 
# * Quanto maior o valor do seguro incendio maior o valor do aluguel.

In [ ]:
# # Existe qualquer relação entre o valor do iptu e o valor do aluguel?

sns.jointplot(y= data['valor_aluguel'],x = data['valor_iptu'],kind ='reg')

# * Não temos nenhum impacto

In [ ]:
# **Pré-processando os dados**
# 
# Poucos algoritmos trabalham com dados categóricos, por essas variáveis serem importantes para o nosso modelo precisamos transformar dados categóricos em dados numéricos para poder usar os algoritmos de Machine Learning
# 
# Podemos usar duas técnicas bem eficientes que são:
# 
# - **Label Encoder**
# - **One Hot Enconder**

# Verificando a quantidade de valores únicos na coluna estado

data.estado.unique()

In [ ]:
# Verificando a quantidade de valores únicos na coluna cidade

data.cidade.unique()

data.head()

In [ ]:
# Importando a biblioteca

# **Aplicando o One Hot Encoder**

# Subistituindo o dataframe data pelo dataframe após aplicado o get_dummies()
#data = pd.get_dummies(data, columns=["estado","cidade"], prefix=["estado_","cidade_"])

# Subistituindo o dataframe data pelo dataframe após aplicado o dicionarios

subs = {'São Paulo':1, 'Porto Alegre':2, 'Rio de Janeiro':3, 'Campinas':4, 'Belo Horizonte':5}
data['cidade'] = data['cidade'].replace(subs)

subs = {'SP':1, 'RS':2, 'RJ':3, 'MG':4}
data['estado'] = data['estado'].replace(subs)


data.head()

In [ ]:
# Aula 03

# # Detecção de Outliers, Machine Learning

# Excluindo os atributos categoricos


data.drop(["animal_categorical","mobilia_categorical"], axis=1, inplace=True)

In [ ]:
#Outlier Detection

#Verificando os registros anomalos no atributo valor_aluguel

plt.figure(figsize=(8, 4))
sns.boxplot(x=data['valor_aluguel'])
plt.show()

In [ ]:
#Quantil 1 = 25% dos dados.


Q1 = data["valor_aluguel"].quantile(.25)

#Quantil 3 = 75% dos dados.

Q3 = data["valor_aluguel"].quantile(.75)

Q1,Q3

In [ ]:
#Faixa de Intervalo Interqualítico

IQR = Q3 - Q1

IQR

In [ ]:
#Calculando os limites

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

lower,upper

In [ ]:
#Definindo a seleção dos registros

data_clean = data[data['valor_aluguel'] >= lower] 

data_clean = data_clean[data_clean['valor_aluguel'] <= upper]

data_clean.describe()

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.title('Com Outliers')
sns.boxplot(x= data['valor_aluguel'])

plt.subplot(1, 2, 2)
plt.title('Sem Outliers')
sns.boxplot(x= data_clean['valor_aluguel'])

plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
data = data_clean

# # Machine Learning

#Separando os dados em features e classes.

y = data['valor_aluguel']
X = data.drop(['valor_aluguel'],axis =1)

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
#Separando os conjuntos de treino e teste.

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calcula_metricas(y_test,preds):
    print('MAE:', mean_absolute_error(y_test, preds))
    print('RMSE:', np.sqrt(mean_squared_error(y_test, preds)))
    print('R2:', r2_score(y_test, preds))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.3)

X_train

In [ ]:
acc=[]

#**Calculando a nossa Baseline**

from sklearn.dummy import DummyRegressor

model = DummyRegressor()

# Iniciando o treinamento.

model.fit(X_train,y_train)

In [ ]:
#Fazendo as predições

preds = model.predict(X_test)

#Calculando as métricas.

calcula_metricas(y_test,preds)

In [ ]:
#Armazendo o resultado.

acc.append(
            [
              'BAS', r2_score(y_test,preds)
            ]
          )

acc

In [ ]:
# #### Treinando um modelo de Regressão Linear

#Instanciando o estimator.

from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [ ]:
#Iniciando o treinamento.

model.fit(X_train,y_train)

# Fazendo as predições

preds = model.predict(X_test)

In [ ]:
#Salvando o modelo Linear_Regression 

import joblib 

print(joblib.__version__)

filename = 'modelo-final-LR.pkl'
joblib.dump(model, filename)

In [ ]:
#Calculando as métricas.


calcula_metricas(y_test,preds)


#Armazendo o resultado.

acc.append(
            [
              'LIR', r2_score(y_test,preds)
            ]
          )

acc

In [ ]:
loaded_model = joblib.load(filename)

#Fazendo as predições

preds = loaded_model.predict(X_test)

#Calculando as métricas.

calcula_metricas(y_test, preds)

#Imprimie o Resultado.

print(r2_score(y_test,model.predict(X_test)))

In [ ]:
# Decision Tree Regression

from sklearn.tree import DecisionTreeRegressor

#Iniciando o treinamento.

model = DecisionTreeRegressor()
model.fit(X_train,y_train)

# Fazendo as predições

preds = model.predict(X_test)


#Calculando as métricas.

calcula_metricas(y_test,preds)

#Armazendo o resultado.


acc.append(['DTR',r2_score(y_test,preds)])

acc

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestRegressor

#Iniciando o treinamento.

model = RandomForestRegressor()
model.fit(X_train,y_train)

#Fazendo as predições

preds = model.predict(X_test)

#Calculando as métricas.

calcula_metricas(y_test,preds)

#Armazendo o resultado.

acc.append(['RFN',r2_score(y_test,model.predict(X_test))])

acc

In [ ]:
import sklearn

sklearn.__version__

In [ ]:
#Salvando o modelo RandomForest 

import joblib 

print(joblib.__version__)

filename = 'modelo-final-RF.pkl'
joblib.dump(model, filename)

In [ ]:
loaded_model = joblib.load(filename)

#Fazendo as predições

preds = loaded_model.predict(X_test)

#Calculando as métricas.

calcula_metricas(y_test, preds)

#Imprimie o Resultado.

print(r2_score(y_test,model.predict(X_test)))

In [ ]:
#KNN Regressor

from sklearn.neighbors import KNeighborsRegressor

#Iniciando o treinamento.

model = KNeighborsRegressor()
model.fit(X_train,y_train)

#Fazendo as predições

preds = model.predict(X_test)

#Calculando as métricas.

calcula_metricas(y_test,preds)

#Armazendo o resultado.

acc.append(['KNNR',r2_score(y_test,model.predict(X_test))])

acc

In [ ]:
#Ordenando os resultados.

acc.sort(key = lambda y:y[1],reverse =True)

acc